In [6]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 18 10:19:34 2021

@author: Valen
"""
#Hello

import sys
sys.path.insert(0, 'evoman')
from environment import Environment
from demo_controller import player_controller

# imports other libs
import time
import numpy as np
#from math import fabs,sqrt
import glob, os

# choose this for not using visuals and thus making experiments faster
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"


experiment_name = 'individual_demo'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

n_hidden_neurons = 10

# initializes simulation in individual evolution mode, for single static enemy.
env = Environment(experiment_name=experiment_name,
                  enemies=[2],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=2,
                  speed="fastest")

# default environment fitness is assumed for experiment

env.state_to_log() # checks environment state


####   Optimization for controller solution (best genotype-weights for phenotype-network): Ganetic Algorihm    ###

ini = time.time()  # sets time marker


# genetic algorithm params

run_mode = 'train' # train or test

# number of weights for multilayer with 10 hidden neurons
n_vars = (env.get_num_sensors()+1)*n_hidden_neurons + (n_hidden_neurons+1)*5
print("Number of weights: " + str(n_vars))


# runs simulation
def simulation(env,x):
    f,p,e,t = env.play(pcont=x)
    return f

# evaluation
def evaluate(x):
    return np.array(list(map(lambda y: simulation(env,y), x)))

# DEAP - own code starts here 
from deap import base, creator
import random
from deap import tools
#maximizing fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
#creator.create("Individual", list, fitness=creator.FitnessMax)
from array import array
creator.create("Individual", array, typecode="d",
               fitness=creator.FitnessMax, strategy=None)
creator.create("Strategy", array, typecode="d")

def initES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

IND_SIZE = n_vars
MIN_VALUE, MAX_VALUE = -1., 1.
MIN_STRAT, MAX_STRAT = -1., 1. 

toolbox = base.Toolbox()
toolbox.register("individual", initES, creator.Individual,
                 creator.Strategy, IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRAT, 
                 MAX_STRAT)


#uniform = random.uniform(-1, 1)
toolbox.register("attribute", np.random.uniform, -1, 1)
#random.uniform(-1.,1.)
#toolbox.register("individual", tools.initRepeat, creator.Individual,
#                 toolbox.attribute, n= n_vars)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
#toolbox.register("population", tools.initRepeat, np.array([]), toolbox.individual)



toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutESLogNormal, c = 1, indpb=0.1)
toolbox.register("parentselection", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)
toolbox.register("survivalselection", tools.selBest)

#to compile statistics
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)
logbook = tools.Logbook()


pop_size = 20

#test

def main():
    # create variables to later save the best individual
    best_ind = 1
    best_fit = -100
    
    
    pop = toolbox.population(n=pop_size)
    CXPB, MUTPB, NGEN = 0.7, 0.2, 10

    # Evaluate the entire population
    #fitnesses = map(toolbox.evaluate, np.array(pop))
    #for ind, fit in zip(pop, fitnesses):
    #    ind.fitness.values = fit
    for i in range(pop_size):
        fitness = toolbox.evaluate(np.array([pop[i]]))
        pop[i].fitness.values = fitness
        if fitness > best_fit:
            best_fit = fitness
            best_ind = pop[i]

    # show first random generation
    record = stats.compile(pop)
    print("Gen 0: ")
    print(record)
    

    for g in range(NGEN):
        # Select the next generation individuals - parent selection 
        parents = toolbox.parentselection(pop, (2*len(pop)))
        # Clone the selected individuals
        parents = list(map(toolbox.clone, parents))

        
        offspring = []
        # Apply crossover on the offspring
        
        #like this
        for parent1, parent2 in zip(parents[::2], parents[1::2]): 
            if random.random() < CXPB:
                child1, child2 = toolbox.mate(parent1, parent2)
                del child1.fitness.values
                del child2.fitness.values
                offspring.append(child1)
                offspring.append(child2)
        print(len(offspring)) 
        
        #or like this
# =============================================================================
#         for child1, child2 in zip(parents[::2], parents[1::2]):
#             if random.random() < CXPB:
#                 toolbox.mate(child1, child2)
#                 del child1.fitness.values
#                 del child2.fitness.values
# =============================================================================
                
        # mutation
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        #invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        #for i in range(len(invalid_ind)):
        #    invalid_ind[i].fitness.values = toolbox.evaluate(np.array([invalid_ind[i]]))
        for i in range(len(offspring)):
            fitness = toolbox.evaluate(np.array([offspring[i]]))
            offspring[i].fitness.values = fitness
            if fitness > best_fit:
                best_fit = fitness
                best_ind = offspring[i]
            
        
        survivedoffspring = toolbox.survivalselection(offspring, pop_size)
        
        # The population is entirely replaced by the offspring
        pop = survivedoffspring
        print(len(pop))
        
        record = stats.compile(pop)
        print("Gen : " + str(g+1))
        print(record)
        logbook.record(gen= (g +1), evals=30, **record)
        
    return best_ind



MESSAGE: Pygame initialized for simulation.

########## Simulation state - INI ###########

# solutions # : EMPTY 

# sensors # : 20

 ------  parameters ------  

# contact hurt (training agent) # : player

multiple mode: no

enemies list: 2

current enemy: 2

player mode: ai

enemy mode: static

level: 2

clock precision: low

inputs coded: no

random initialization: no

expiration time: 3000

speed: fastest

load player: yes

load enemy: yes

sound: off

overture time: 100

logs: on

save logs: yes

########## Simulation state - END ###########
Number of weights: 265


C:\Users\Valen\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Valen\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Valen\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Strategy' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [7]:
best_individuals = []
for i in range(3):
    best_ind = main()
    best_individuals.append(best_ind)
    
print(logbook)


RUN: run status: enemy: 2; fitness: 39.299556426609314; player life: 0; enemy life: 50.0; time: 299

RUN: run status: enemy: 2; fitness: -5.783825182329737; player life: 0; enemy life: 100; time: 325

RUN: run status: enemy: 2; fitness: -5.84354441703136; player life: 0; enemy life: 100; time: 345

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329

RUN: run status: enemy: 2; fitness: -5.886104031450156; player life: 0; enemy life: 100; time: 360

RUN: run status: enemy: 2; fitness: 3.203942249234628; player life: 0; enemy life: 90.0; time: 329

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329

RUN: run status: enemy: 2; fitness: -5.886104031450156; player life: 0; enemy life: 100; time: 360

RUN: run status: enemy: 2; fitness: 92.13703951986405; player life: 78; enemy life: 0; time: 288

R


RUN: run status: enemy: 2; fitness: 3.575049982518597; player life: 0; enemy life: 90.0; time: 227

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 3.203942249234628; player life: 0; enemy life: 90.0; time: 329

RUN: run status: enemy: 2; fitness: -5.802118375377063; player life: 0; enemy life: 100; time: 331

RUN: run status: enemy: 2; fitness: 92.57611947915372; player life: 76; enemy life: 0; time: 152

RUN: run status: enemy: 2; fitness: 21.234808897215157; player life: 0; enemy life: 70.0; time: 319
20
Gen : 2
{'avg': array([79.49676269]), 'std': array([22.1762009]), 'min': array([30.2634277]), 'max': array([92.93193986])}
26

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 87.72700044143625; player life: 30; enemy


RUN: run status: enemy: 2; fitness: 92.34772657195336; player life: 76; enemy life: 0; time: 191

RUN: run status: enemy: 2; fitness: 12.30964054567594; player life: 0; enemy life: 80.0; time: 296

RUN: run status: enemy: 2; fitness: 92.53109579777976; player life: 76; enemy life: 0; time: 159

RUN: run status: enemy: 2; fitness: 87.72700044143625; player life: 30; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 39.14492807779757; player life: 0; enemy life: 50.0; time: 349

RUN: run status: enemy: 2; fitness: 92.57611947915372; player life: 76; enemy life: 0; time: 152

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237
20
Gen : 5
{'avg': array([90.7941459]), 'std': array([2.27500271]), 'min': array([87.72700044]), 'max': array([92.93193986])}
28

RUN: run status: enemy: 2; fitness: 89.80278772623588; player life: 48; enemy life


RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.64704041507838; player life: 82; enemy life: 0; time: 258

RUN: run status: enemy: 2; fitness: 92.29061166337202; player life: 78; enemy life: 0; time: 247

RUN: run status: enemy: 2; fitness: 66.21310261863329; player life: 0; enemy life: 20.0; time: 326

RUN: run status: enemy: 2; fitness: 92.04704041507837; player life: 76; enemy life: 0; time: 258

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278
20
Gen : 8
{'avg': array([92.67118244]), 'std': array([0.43484468]), 'min': array([91.77237889]), 'max': array([92.93193986])}
34

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life:


RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 12.12506926914797; player life: 0; enemy life: 80.0; time: 356

RUN: run status: enemy: 2; fitness: -5.707110264748875; player life: 0; enemy life: 100; time: 301

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329

RUN: run status: enemy: 2; fitness: 3.2161748176702627; player life: 0; enemy life: 90.0; time: 325

RUN: run status: enemy: 2; fitness: 88.42385026742616; player life: 36; enemy life: 0; time: 177

RUN: run status: enemy: 2; fitness: 48.21925648420767; player life: 0; enemy life: 40.0; time: 324

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: -5.318119993844216; player life: 0; enemy life: 100; time: 204

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329
Gen 0

...


RUN: run status: enemy: 2; fitness: 93.32700044143624; player life: 86; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.52700044143626; player life: 88; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.52700044143626; player life: 88; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 91.72700044143625; player life: 70; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.52700044143626; player life: 88; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.32700044143624; player life: 86; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.52700044143626; player life: 88; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.52700044143626; player life: 88; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 92.52700044143626; player life: 78; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: en


RUN: run status: enemy: 2; fitness: -5.783825182329737; player life: 0; enemy life: 100; time: 325

RUN: run status: enemy: 2; fitness: 30.20394224923463; player life: 0; enemy life: 60.0; time: 329

RUN: run status: enemy: 2; fitness: -5.783825182329737; player life: 0; enemy life: 100; time: 325

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: -5.313205979041787; player life: 0; enemy life: 100; time: 203

RUN: run status: enemy: 2; fitness: -5.313205979041787; player life: 0; enemy life: 100; time: 203

RUN: run status: enemy: 2; fitness: 39.35102576183879; player life: 0; enemy life: 50.0; time: 284

RUN: run status: enemy: 2; fitness: -5.796057750765372; player life: 0; enemy life: 100; time: 329

RUN: run status: enemy: 2; fitness: 21.062463794917573; player life: 0; enemy life: 70.0; time: 379

RUN: run status: enemy: 2; fitness: 87.72700044143625; player life: 30; enemy life: 0; time: 195

RU


RUN: run status: enemy: 2; fitness: 66.75825298494036; player life: 0; enemy life: 20.0; time: 189

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 21.28957298262513; player life: 0; enemy life: 70.0; time: 302

RUN: run status: enemy: 2; fitness: 39.20394224923463; player life: 0; enemy life: 50.0; time: 329

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 92.37237888630936; player life: 80; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 91.77237888630937; player life: 74; enemy life: 0; time: 278

RUN: run status: enemy: 2; fitness: 39.20394224923463; player life: 0; enemy life: 50.0; time: 329

RUN: run status: enemy: 2; fitness: 92.17237888630936; player life: 78; enemy life: 0; time: 278

RUN: run st


RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 74.69555119757801; player life: 0; enemy life: 10.0; time: 547

RUN: run status: enemy: 2; fitness: 93.23521402607649; player life: 84; enemy life: 0; time: 175

RUN: run status: enemy: 2; fitness: 39.20394224923463; player life: 0; enemy life: 50.0; time: 329

RUN: run status: enemy: 2; fitness: 12.502831774706799; player life: 0; enemy life: 80.0; time: 244

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 30.73214184093667; player life: 0; enemy life: 60.0; time: 194

RUN: run status: enemy: 2; fitness: 91.09466846406764; player life: 66; enemy life: 0; time: 246

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run s


RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 86.92700044143626; player life: 22; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.12700044143625; player life: 84; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 48.76355803717005; player life: 0; enemy life: 40.0; time: 188

RUN: run status: 

In [8]:
for indiv in best_individuals:
    #print(indiv)
    print(indiv.fitness.values)
    

(92.93193985886488,)
(93.72700044143625,)
(93.23521402607649,)


In [10]:
means = []
for indiv in best_individuals:
    fiveruns = []
    for i in range(5):
        fitness = toolbox.evaluate(np.array([indiv]))
        fiveruns.append(fitness)
    means.append(np.mean(fiveruns))
means
        
        
    


RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 92.93193985886488; player life: 84; enemy life: 0; time: 237

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: enemy: 2; fitness: 93.72700044143625; player life: 90; enemy life: 0; time: 195

RUN: run status: en

[92.93193985886488, 93.72700044143625, 93.23521402607649]

In [ ]:
import matplotlib.pyplot as plt



In [5]:
# create variables to later save the best individual
best_ind = 1
best_fit = -100
    

pop = toolbox.population(n=pop_size)
CXPB, MUTPB, NGEN = 0.7, 0.2, 10

# Evaluate the entire population
#fitnesses = map(toolbox.evaluate, np.array(pop))
#for ind, fit in zip(pop, fitnesses):
#    ind.fitness.values = fit
for i in range(pop_size):
    fitness = toolbox.evaluate(np.array([pop[i]]))
    pop[i].fitness.values = fitness
    if fitness > best_fit:
        best_fit = fitness
        best_ind = pop[i]
        
print(best_ind)
print(best_ind.fitness.values)


RUN: run status: enemy: 8; fitness: -5.293304824724492; player life: 0; enemy life: 100; time: 199

RUN: run status: enemy: 8; fitness: 3.732141840936672; player life: 0; enemy life: 90.0; time: 194

RUN: run status: enemy: 8; fitness: 3.7066951752755077; player life: 0; enemy life: 90.0; time: 199

RUN: run status: enemy: 8; fitness: 3.721885340769483; player life: 0; enemy life: 90.0; time: 196

RUN: run status: enemy: 8; fitness: -5.442417710521793; player life: 0; enemy life: 100; time: 231

RUN: run status: enemy: 8; fitness: -5.75890177387728; player life: 0; enemy life: 100; time: 317

RUN: run status: enemy: 8; fitness: -5.393627546352362; player life: 0; enemy life: 100; time: 220

RUN: run status: enemy: 8; fitness: 3.575049982518597; player life: 0; enemy life: 90.0; time: 227

RUN: run status: enemy: 8; fitness: 21.059828747279568; player life: 0; enemy life: 70.0; time: 380

RUN: run status: enemy: 8; fitness: -5.75890177387728; player life: 0; enemy life: 100; time: 317
